In [1]:
import polars as pl

In [2]:
# Fetch data from clickhouse
!curl -u aurelion_admin:test1234 -sS 'http://localhost:18123/' --data-binary "SELECT * FROM aurelion.features FORMAT Parquet" -o ../data/features.parquet


In [3]:
# Load fetched data
df = pl.read_parquet("../data/features.parquet", use_pyarrow=True)
df

event_time,instrument,feature_type,feature_id,key,value
u32,str,str,str,str,f64
1722196800,"""BTC""","""PCA""","""PCA""","""COMPONENT_0""",12.0
1722196860,"""BTC""","""PCA""","""PCA""","""COMPONENT_0""",13.0
1722196920,"""BTC""","""PCA""","""PCA""","""COMPONENT_0""",10.0
1722196800,"""BTC""","""PCA""","""PCA""","""COMPONENT_1""",32.0
1722196860,"""BTC""","""PCA""","""PCA""","""COMPONENT_1""",30.0
1722196920,"""BTC""","""PCA""","""PCA""","""COMPONENT_1""",29.0
1722196800,"""BTC""","""SMA""","""SMA_5""","""SMA_5""",5.0
1722196860,"""BTC""","""SMA""","""SMA_5""","""SMA_5""",6.0
1722196920,"""BTC""","""SMA""","""SMA_5""","""SMA_5""",15.0
